In [1]:
%%capture
# install libraries
%pip install pickle 
%pip install pandas 
%pip install beautifulsoup4
%pip install selenium
%pip install webdriver-manager
%pip install lxml

In [2]:
import os
import time
import pickle
import pandas as pd
from bs4 import BeautifulSoup as BS
from webdriver_manager.firefox import GeckoDriverManager
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from datetime import datetime
import re

import warnings
warnings.filterwarnings('ignore')

In [3]:
def start_driver():
    caps = DesiredCapabilities().FIREFOX
    #caps["pageLoadStrategy"] = "eager"
    driver = webdriver.Firefox(service=Service(executable_path=GeckoDriverManager().install(), desired_capabilities = caps))
    driver.maximize_window()
    return driver

def get_info(driver, link, sleeptime=2):
    driver.get(link)
    time.sleep(sleeptime)
    content = driver.page_source
    return BS(content, 'lxml')

In [4]:
link = "https://www.amazon.in/s?i=stripbooks&rh=p_27%3ASteven+S+Skiena&ref=dp_byline_sr_book_1"
driver = start_driver()
soup = get_info(driver = driver, link = link)

In [5]:
data = []
books = soup.find('div', 's-main-slot s-result-list s-search-results sg-row').find_all('div', 'puis-card-container')
for idx, book in enumerate(books, start = 1):
    
    # title
    title = book.find('span', 'a-size-medium').text.strip()
    
    # Published date
    published_date = book.find('span', 'a-size-base a-color-secondary a-text-normal').text.strip()
    
    # Hardcover, Kindle, and Paperback prices
    hardcover_price = kindle_price = paperback_price = None
    is_hardcover = 'Hardcover' in str(book)
    is_kindle = 'Kindle Edition' in str(book)
    is_paperback = 'Paperback' in str(book)

    if is_hardcover:
        hardcover_price = book.find_all('span', 'a-price')[0].find('span', 'a-price-whole').text.strip()
        hardcover_price = float(hardcover_price.replace(',', ''))
        
    if is_paperback:
        try:
            paperback_price = book.find_all('span', 'a-price')[-2].find('span', 'a-price-whole').text.strip()
            paperback_price = float(paperback_price.replace(',', ''))
        except:
            pass
        
    if is_kindle:
        try:
            index = 2 if is_hardcover else 0
            kindle_price = book.find_all('span', 'a-price')[index].find('span', 'a-price-whole').text.strip()
            kindle_price = float(kindle_price.replace(',', ''))
        except:
            kindle_price = book.find_all('span', 'a-price')[0].find('span', 'a-price-whole').text.strip()
            kindle_price = float(kindle_price.replace(',', ''))
            
            hardcover_price = book.find_all('span', 'a-price')[1].find('span', 'a-price-whole').text.strip()
            hardcover_price = float(hardcover_price.replace(',', ''))
            
            paperback_price = book.find_all('span', 'a-price')[-2].find('span', 'a-price-whole').text.strip()
            paperback_price = float(paperback_price.replace(',', ''))
        finally:
            pass
        
    # rating
    rating = float(book.find('div', 'a-row a-size-small').find('span')['aria-label'].split(' ')[0])
    
    # users rated
    users_rated = float(book.find('div', 'a-row a-size-small').find_all('span')[-1].text.strip())
    
    # isbn
    
    data.append({
        'rank'             : idx,
        'title'             : title,
        'published_date'    : published_date,
        'hardcover_price'   : hardcover_price,
        'kindle_price'      : kindle_price,
        'paperback_price'   : paperback_price,
        'rating'            : rating,
        'users_rated'       : users_rated
    })

In [6]:
df = pd.DataFrame(data)
df

,rank,title,published_date,hardcover_price,kindle_price,paperback_price,rating,users_rated
0,1,The Algorithm Design Manual (Texts in Computer...,6 October 2020,2800.0,4235.0,3918.0,4.5,338.0
1,2,The Data Science Design Manual (Texts in Compu...,29 August 2017,4054.0,5192.0,3851.0,4.6,77.0
2,3,The Algorithm Design Manual,5 April 2009,3403.0,4000.0,4000.0,4.4,572.0
3,4,Programming Challenges: The Programming Contes...,18 April 2006,NaN,5239.0,5239.0,4.3,31.0
4,5,Desafíos de programación: El manual de entrena...,14 June 2020,NaN,0.0,3303.0,5.0,11.0
5,6,"Calculated Bets: Computers, Gambling, and Math...",6 August 2001,3707.0,760.0,2267.0,4.2,30.0
6,7,The Algorithm Design Manual,1 January 2007,NaN,NaN,2599.0,5.0,1.0
7,8,Desafíos de programación,5 April 2012,NaN,NaN,NaN,5.0,2.0


In [7]:
df.to_csv('steven_skiena_books_amazon.csv', index = False)

In [8]:
driver.quit()